# TNT Loop-Bridge: SAOS

## Objectives

- Fit TNT Loop-Bridge model to Small Amplitude Oscillatory Shear (SAOS) data
- Understand linear viscoelastic response with constant bridge fraction
- Analyze effective Maxwell behavior at equilibrium
- Quantify storage and loss moduli from loop-bridge kinetics
- Perform Bayesian inference for parameter uncertainty

## Setup

In [ ]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTLoopBridge

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_ml_ikh_flow_curve,
    load_pnas_startup,
    load_laponite_relaxation,
    load_ml_ikh_creep,
    load_epstein_saos,
    load_pnas_laos,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_loop_bridge_param_names,
    plot_loop_bridge_fraction,
    plot_bell_nu_sweep,
    compute_maxwell_moduli,
    compute_bell_effective_lifetime,
    print_nu_interpretation,
)

param_names = get_tnt_loop_bridge_param_names()

from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

## Theory: SAOS Response

### Physical Picture

In Small Amplitude Oscillatory Shear (SAOS):
- Strain amplitude is small (typically < 1%)
- Linear response regime
- Bridge fraction remains constant at equilibrium: f_B = f_B_eq
- No force-enhanced detachment (shear rate oscillates around zero)

### Governing Equations

**Effective Maxwell Model:**

At equilibrium (f_B = f_B_eq), the model reduces to a Maxwell element:
```
G_eff = f_B_eq * G
tau_eff = tau_b
```

**Storage Modulus (elastic):**
```
G'(ω) = G_eff * (ω * tau_b)² / [1 + (ω * tau_b)²]
```

**Loss Modulus (viscous):**
```
G''(ω) = G_eff * (ω * tau_b) / [1 + (ω * tau_b)²] + η_s * ω
```

**Complex Modulus:**
```
G*(ω) = G'(ω) + i * G''(ω)
|G*| = sqrt(G'² + G''²)
```

### Frequency Regimes

1. **Low frequency (ω << 1/tau_b):**
   - G' ~ G_eff * (ω * tau_b)² (elastic, terminal regime)
   - G'' ~ G_eff * (ω * tau_b) + η_s * ω (viscous dominant)
   - G'' > G' (liquid-like)

2. **Crossover (ω ~ 1/tau_b):**
   - G' = G'' (loss tangent = 1)
   - Defines characteristic relaxation frequency

3. **High frequency (ω >> 1/tau_b):**
   - G' ~ G_eff (elastic plateau)
   - G'' ~ η_s * ω (solvent viscosity)
   - G' > G'' (solid-like)

### Key Features

- **Plateau modulus**: G_eff = f_B_eq * G
- **Crossover frequency**: ω_c = 1 / tau_b
- **Solvent contribution**: G'' increases linearly at high ω
- **Bridge fraction**: Determines plateau height (higher f_B_eq → higher G_eff)

## Load SAOS Data

In [ ]:
omega, G_prime, G_double_prime = load_epstein_saos()

# Compute complex modulus magnitude
G_star_mag = np.sqrt(G_prime**2 + G_double_prime**2)

print(f"Data points: {len(omega)}")
print(f"Frequency range: {omega.min():.2e} - {omega.max():.2e} rad/s")
print(f"G' range: {G_prime.min():.2e} - {G_prime.max():.2e} Pa")
print(f"G'' range: {G_double_prime.min():.2e} - {G_double_prime.max():.2e} Pa")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# G' and G'' vs omega
ax1.loglog(omega, G_prime, 'o', label="G' (Storage)", markersize=6)
ax1.loglog(omega, G_double_prime, 's', label="G'' (Loss)", markersize=6)
ax1.set_xlabel('Frequency ω (rad/s)', fontsize=12)
ax1.set_ylabel('Modulus (Pa)', fontsize=12)
ax1.set_title('SAOS Data', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Loss tangent
tan_delta = G_double_prime / G_prime
ax2.semilogx(omega, tan_delta, 'o', markersize=6)
ax2.axhline(1.0, color='r', linestyle='--', alpha=0.5, label='tan(δ) = 1')
ax2.set_xlabel('Frequency ω (rad/s)', fontsize=12)
ax2.set_ylabel('tan(δ) = G\"/G\'', fontsize=12)
ax2.set_title('Loss Tangent', fontsize=14)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.close("all")
plt.close('all')

## NLSQ Fitting

In [ ]:
model = TNTLoopBridge()

print("Starting NLSQ fit...")
t_start = time.time()

nlsq_result = model.fit(omega, G_star_mag, test_mode='oscillation', method='scipy')

t_nlsq = time.time() - t_start
print(f"\nNLSQ fit completed in {t_nlsq:.2f} seconds")
print(f"\nFitted parameters:")
for name in param_names:
    value = model.parameters.get_value(name)
    print(f"  {name}: {value:.4e}")

G_star_mag_pred_fit = model.predict(omega, test_mode='oscillation')
metrics = compute_fit_quality(G_star_mag, G_star_mag_pred_fit)
print(f"\nFit quality:")
print(f"  R²: {metrics['R2']:.6f}")
print(f"  RMSE: {metrics['RMSE']:.4e}")
# print(f"  Max relative error: {metrics['max_rel_error']:.2f}%")

## NLSQ Fit Visualization

In [ ]:
omega_pred = jnp.logspace(jnp.log10(omega.min()), jnp.log10(omega.max()), 200)
G_prime_pred, G_double_prime_pred = model.predict_saos(omega_pred)
G_star_pred = jnp.sqrt(G_prime_pred**2 + G_double_prime_pred**2)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Moduli fit
ax1.loglog(omega, G_prime, 'o', label="G' Data", markersize=6, alpha=0.7)
ax1.loglog(omega, G_double_prime, 's', label="G'' Data", markersize=6, alpha=0.7)
ax1.loglog(omega_pred, G_prime_pred, '-', label="G' Fit", linewidth=2)
ax1.loglog(omega_pred, G_double_prime_pred, '-', label="G'' Fit", linewidth=2)
ax1.set_xlabel('Frequency ω (rad/s)', fontsize=12)
ax1.set_ylabel('Modulus (Pa)', fontsize=12)
ax1.set_title(f'SAOS Fit (R² = {metrics["R2"]:.4f})', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Residuals
G_star_fit = model.predict(omega, test_mode='oscillation')
residuals = (G_star_mag - G_star_fit) / G_star_mag * 100
ax2.semilogx(omega, residuals, 'o', markersize=6)
ax2.axhline(0, color='k', linestyle='--', alpha=0.3)
ax2.set_xlabel('Frequency ω (rad/s)', fontsize=12)
ax2.set_ylabel('Relative Error (%)', fontsize=12)
ax2.set_title('Fit Residuals', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.close("all")
plt.close('all')

## Physical Analysis: Effective Maxwell Parameters

In [ ]:
# Compute effective Maxwell parameters
G_eff = model.parameters.get_value('f_B_eq') * model.parameters.get_value('G')
tau_eff = model.parameters.get_value('tau_b')
omega_c = 1.0 / tau_eff

# Maxwell predictions
G_prime_maxwell, G_double_prime_maxwell = compute_maxwell_moduli(omega_pred, G_eff, tau_eff)
G_double_prime_maxwell_with_solvent = G_double_prime_maxwell + model.parameters.get_value('eta_s') * omega_pred

fig, ax = plt.subplots(figsize=(10, 7))
ax.loglog(omega_pred, G_prime_pred, '-', label="G' (Model)", linewidth=2)
ax.loglog(omega_pred, G_double_prime_pred, '-', label="G'' (Model)", linewidth=2)
ax.loglog(omega_pred, G_prime_maxwell, '--', label="G' (Maxwell)", linewidth=2, alpha=0.7)
ax.loglog(omega_pred, G_double_prime_maxwell_with_solvent, '--', label="G'' (Maxwell + solvent)", linewidth=2, alpha=0.7)
ax.axvline(omega_c, color='purple', linestyle=':', alpha=0.5, label=f'ω_c = {omega_c:.4e} rad/s')
ax.axhline(G_eff, color='r', linestyle=':', alpha=0.5, label=f'G_eff = {G_eff:.4e} Pa')
ax.set_xlabel('Frequency ω (rad/s)', fontsize=12)
ax.set_ylabel('Modulus (Pa)', fontsize=12)
ax.set_title('Loop-Bridge vs Effective Maxwell', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.close("all")
plt.close('all')

print(f"\nEffective Maxwell parameters:")
print(f"  G_eff = f_B_eq * G: {G_eff:.4e} Pa")
print(f"  tau_eff = tau_b: {tau_eff:.4e} s")
print(f"  Crossover frequency ω_c: {omega_c:.4e} rad/s")
print(f"  Bridge fraction f_B_eq: {model.parameters.get_value('f_B_eq'):.4f}")
print(f"  Plateau modulus G: {model.parameters.get_value('G'):.4e} Pa")

## Physical Analysis: Crossover Frequency

In [ ]:
# Find crossover where G' = G''
diff = jnp.abs(G_prime_pred - G_double_prime_pred)
crossover_idx = jnp.argmin(diff)
omega_crossover = omega_pred[crossover_idx]
G_crossover = G_prime_pred[crossover_idx]

fig, ax = plt.subplots(figsize=(10, 7))
ax.loglog(omega_pred, G_prime_pred, '-', label="G'", linewidth=2)
ax.loglog(omega_pred, G_double_prime_pred, '-', label="G''", linewidth=2)
ax.plot(omega_crossover, G_crossover, 'ro', markersize=12, label=f'Crossover: ω = {omega_crossover:.4e} rad/s')
ax.axvline(omega_c, color='purple', linestyle='--', alpha=0.5, label=f'ω_c (theory) = {omega_c:.4e} rad/s')
ax.set_xlabel('Frequency ω (rad/s)', fontsize=12)
ax.set_ylabel('Modulus (Pa)', fontsize=12)
ax.set_title('Crossover Frequency Analysis', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.close("all")
plt.close('all')

print(f"\nCrossover analysis:")
print(f"  Measured crossover ω: {omega_crossover:.4e} rad/s")
print(f"  Theoretical ω_c = 1/tau_b: {omega_c:.4e} rad/s")
print(f"  Ratio (measured/theory): {omega_crossover / omega_c:.4f}")
print(f"  Modulus at crossover: {G_crossover:.4e} Pa")
print(f"  G_eff / 2 (Maxwell prediction): {G_eff / 2:.4e} Pa")

## Physical Analysis: Frequency Regimes

In [ ]:
# Define regime boundaries
omega_low = omega_c / 10
omega_high = omega_c * 10

# Extract values in each regime
idx_low = jnp.argmin(jnp.abs(omega_pred - omega_low))
idx_high = jnp.argmin(jnp.abs(omega_pred - omega_high))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# G' vs ω showing regimes
ax1.loglog(omega_pred, G_prime_pred, '-', linewidth=2, label="G'")
ax1.axvline(omega_low, color='r', linestyle='--', alpha=0.5, label='Low ω regime')
ax1.axvline(omega_high, color='g', linestyle='--', alpha=0.5, label='High ω regime')
ax1.set_xlabel('Frequency ω (rad/s)', fontsize=12)
ax1.set_ylabel('G\' (Pa)', fontsize=12)
ax1.set_title('Storage Modulus Regimes', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Loss tangent
tan_delta_pred = G_double_prime_pred / G_prime_pred
ax2.semilogx(omega_pred, tan_delta_pred, '-', linewidth=2)
ax2.axhline(1.0, color='r', linestyle='--', alpha=0.5, label='tan(δ) = 1')
ax2.axvline(omega_c, color='purple', linestyle='--', alpha=0.5, label='ω_c')
ax2.set_xlabel('Frequency ω (rad/s)', fontsize=12)
ax2.set_ylabel('tan(δ)', fontsize=12)
ax2.set_title('Loss Tangent', fontsize=14)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.close("all")
plt.close('all')

print(f"\nFrequency regime analysis:")
print(f"\n1. Low frequency (ω << ω_c):")
print(f"   ω = {omega_pred[idx_low]:.4e} rad/s")
print(f"   G' = {G_prime_pred[idx_low]:.4e} Pa")
print(f"   G'' = {G_double_prime_pred[idx_low]:.4e} Pa")
print(f"   tan(δ) = {tan_delta_pred[idx_low]:.4f}")
print(f"   Regime: Liquid-like (G'' > G')")

print(f"\n2. High frequency (ω >> ω_c):")
print(f"   ω = {omega_pred[idx_high]:.4e} rad/s")
print(f"   G' = {G_prime_pred[idx_high]:.4e} Pa")
print(f"   G'' = {G_double_prime_pred[idx_high]:.4e} Pa")
print(f"   tan(δ) = {tan_delta_pred[idx_high]:.4f}")
print(f"   Regime: Solid-like (G' > G'')")
print(f"   G' / G_eff = {G_prime_pred[idx_high] / G_eff:.4f}")

## Bayesian Inference

In [ ]:
# FAST_MODE: Use reduced MCMC for quick validation
# FAST_MODE controls Bayesian inference (env var FAST_MODE, default=1)
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    bayes_result = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    bayes_result = model.fit_bayesian(
        omega, G_star_mag,
        test_mode='oscillation',
        
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


## Convergence Diagnostics

In [ ]:
# Skip convergence diagnostics in CI mode
if not FAST_MODE:
    print_convergence_summary(bayes_result, param_names)
else:
    print("FAST_MODE: Skipping convergence diagnostics")


## Parameter Comparison: NLSQ vs Bayesian

In [ ]:
print_parameter_comparison(model, bayes_result.posterior_samples, param_names)

## ArviZ Diagnostics

In [ ]:
# ArviZ diagnostics (trace, pair, forest, energy, autocorrelation, rank)
if not FAST_MODE and hasattr(bayes_result, 'to_inference_data'):
    display_arviz_diagnostics(bayes_result, param_names, fast_mode=FAST_MODE)
else:
    print("FAST_MODE: Skipping ArviZ diagnostics")

## Posterior Predictive

In [ ]:
posterior = bayes_result.posterior_samples
n_draws = 200
indices = np.random.choice(NUM_SAMPLES, size=n_draws, replace=False)

predictions = []
for i in indices:
    # Set parameters from posterior sample
    for name in param_names:
        model.parameters.set_value(name, float(posterior[name][i]))
    # Use predict method
    pred = model.predict(omega_pred, test_mode='oscillation')
    predictions.append(np.array(pred))

predictions = np.array(predictions)
pred_mean = predictions.mean(axis=0)
pred_lower = np.percentile(predictions, 2.5, axis=0)
pred_upper = np.percentile(predictions, 97.5, axis=0)

fig, ax = plt.subplots(figsize=(10, 7))
ax.loglog(omega, G_star_mag, 'o', label='Data', markersize=6, alpha=0.7, zorder=3)
ax.loglog(omega_pred, pred_mean, '-', label='Posterior Mean', linewidth=2, zorder=2)
ax.fill_between(omega_pred, pred_lower, pred_upper, alpha=0.3, label='95% Credible Interval', zorder=1)
ax.set_xlabel('Frequency ω (rad/s)', fontsize=12)
ax.set_ylabel('|G*| (Pa)', fontsize=12)
ax.set_title('Posterior Predictive Distribution', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.close("all")
plt.close('all')

## Physical Interpretation

In [ ]:
print("\n=== Physical Interpretation ===")
print(f"\n1. Material Properties:")
print(f"   - Plateau modulus G: {model.parameters.get_value('G'):.4e} Pa")
print(f"   - Equilibrium bridge fraction: {model.parameters.get_value('f_B_eq'):.4f}")
print(f"   - Effective modulus G_eff: {G_eff:.4e} Pa")
print(f"   - Modulus reduction: {(1 - G_eff/model.parameters.get_value('G'))*100:.2f}%")

print(f"\n2. Maxwell Characteristics:")
print(f"   - Relaxation time tau_b: {model.parameters.get_value('tau_b'):.4e} s")
print(f"   - Crossover frequency ω_c: {omega_c:.4e} rad/s")
print(f"   - Crossover frequency (Hz): {omega_c / (2 * np.pi):.4e} Hz")

print(f"\n3. Frequency Response:")
print(f"   - Low ω: Liquid-like (G'' > G')")
print(f"   - ω ~ ω_c: Crossover (G' = G'')")
print(f"   - High ω: Solid-like (G' > G'')")

print(f"\n4. Solvent Contribution:")
print(f"   - Solvent viscosity η_s: {model.parameters.get_value('eta_s'):.4e} Pa·s")
G_double_prime_high = G_double_prime_pred[idx_high]
solvent_contrib = model.parameters.get_value('eta_s') * omega_pred[idx_high]
print(f"   - G'' at high ω: {G_double_prime_high:.4e} Pa")
print(f"   - Solvent contribution: {solvent_contrib:.4e} Pa ({solvent_contrib/G_double_prime_high*100:.2f}%)")

print(f"\n5. Bridge Kinetics:")
print(f"   - Bridge detachment time tau_b: {model.parameters.get_value('tau_b'):.4e} s")
print(f"   - Loop attachment time tau_a: {model.parameters.get_value('tau_a'):.4e} s")
print(f"   - Ratio tau_a/tau_b: {model.parameters.get_value('tau_a')/model.parameters.get_value('tau_b'):.4f}")
print(f"   - Equilibrium f_B_eq = 1/(1 + tau_a/tau_b): {model.parameters.get_value('f_B_eq'):.4f}")

print(f"\n6. Linear Regime Validity:")
print(f"   - SAOS assumes f_B = f_B_eq (constant)")
print(f"   - Nu parameter: {model.parameters.get_value('nu'):.4f}")
print(f"   - At small strain, Bell detachment negligible")
print(f"   - Model reduces to effective Maxwell")

## Save Results

In [ ]:
save_tnt_results(model, bayes_result, "loop_bridge", "saos", param_names)
print("Results saved to reference_outputs/tnt/loop_bridge_saos_results.npz")

## Key Takeaways

1. **Linear Response**: SAOS probes equilibrium state where f_B = f_B_eq (constant)

2. **Effective Maxwell**: Model reduces to Maxwell element with G_eff = f_B_eq * G, tau_eff = tau_b

3. **Bridge Fraction**: Determines plateau modulus height (lower f_B_eq → lower G_eff)

4. **Crossover Frequency**: ω_c = 1/tau_b defines transition from liquid-like to solid-like

5. **Solvent Viscosity**: Contributes to G'' at high frequencies, linear in ω

6. **Kinetic Balance**: tau_a/tau_b ratio controls f_B_eq via equilibrium condition

7. **No Bell Physics**: At small strain, force-enhanced detachment is negligible (nu irrelevant)